In [9]:
""" Basic Simplex Implementation using tabular form
    Operations Research Workflow by Hiller 
    Note: This is only going to work with a well formed problem. I'm not going to waste time making it work with an illformed problem,
    but that would be the logical next step. 
    This was simply made to help me understand Chapter 4 of the Operations Research 4.4 conceptually. 
    I've not tailored this for optimization, because I want it to be as clean and clear as possible. 
""" 

import numpy as np
from numpy import *
import sys


class SimplexMethod:
    
    """ Loop the solve method until it is finished """
    def solver(self):
        while not self.isFinished():
            self.solve()
        return(self.mat)
    
    """ Solve the current initiated matrix """
    def solve(self):
        self.determine_entering_basic_variable()
        self.minimum_ratio_test()
        self.solve_for_bf()
    
    
    """
    Initalize the current matrix. 
    The left column is Z 
    The middle columns are x1 to x..j, introducing slack variables for each x
    In the example case, there are 4 equations and 6 variables, introducing 2 degrees of freedom.
    """
    def __init__(self, matrix):
        self.mat = matrix
        self.iteration = 0
    
    """ 
        Introduce slack variables in the handed matrix defined. 
        Determine the entering basic variable by looking at the minimum value/column of the tabular matrix. 
    """
    def determine_entering_basic_variable(self):
        #Note: We have issues here is there is equal - coefficient or if there is no negative coefficient. Not checking this case.  
        self.min_coeff = self.mat.min()
        for val in self.mat[0, ]:
            m = reshape(self.mat[0,], (self.mat.shape[1],1))
            for idx, val in enumerate(m):
                if val == self.min_coeff:
                    self.pivot_column_idx = idx

    """ 
    Finds the minimum ratio: Steps:
        1) Pick out each coeff in the pivot column that is positive
        2) Divide these coeff by the right side entry of the same row
        3) Identify the row that has the smallest of these ratios
        4) The basic variable for that row is the leaving basic variable column of the next simplex tableau.
    
    """
    def minimum_ratio_test(self):
        ratio = float('inf')
        self.pivot_column = self.mat[1:,self.pivot_column_idx]
        self.right_column = self.mat[1:,self.mat.shape[1]- 1]
        for idx, row in enumerate(self.pivot_column):
            if(row > 0):
                c_ratio = self.right_column[idx] / row
                if(c_ratio <  ratio):
                    ratio =  c_ratio
                    self.pivot_row_idx = idx + 1 # we need to add 1, because we took out the idx before
        
    """ 
    Solve for the new BF solution by using elementary row operations 
    to construct a new simplex tableau in proper form Gaussian elimanation
    below the current one, then return to the optiality test
    
    Step 
    1) Divide the pivot row by the pivot number
    2) For each row, that has a negative coeff in the pivot column,
    add to this row the product of the abs value of this coeff and the new pivot row
    3) For each other raow that has a positive coeff in the pivot column, subtract from 
    this row the product of this coeff in the new pivot row. 
    Note: There's probably a better "numpy" way to do these iterations, but I'm not sure. In R, it would probably
    be using the apply function. 
    """
    def solve_for_bf(self):
        print('----------------Print solving for ', self.iteration)
        """Level 2 of optimal solution programming"""
        self.iteration += 1
        iter2 = np.empty(self.mat.shape)
        num = self.get_pivot_number()
        new_row = self.mat[self.pivot_row_idx, ] / num #create a new row, with the pivot row/ the pivot number
        
        
        """
        Loop through each row
        if the coeff of the pivot row < 0, add the itermediary matrix the row of the matrix.
        if the coeff of the pivot row is < 0, subtract the itermediary matrix to the row of the matrix
        """
        for idx, row in enumerate(self.mat):
            iter2[idx] = np.abs(self.mat[idx, self.pivot_column_idx]) * new_row #Note. You need to take the abs of the pivot num, but NOT the new row. I made a mistake on this, and took me a while to figure out this was the problem 
       
        #Now we have an addition matrix
        result = np.zeros(iter2.shape)
        for idx, row in enumerate(self.mat):
            coeff = row[self.pivot_column_idx]
            if(coeff < 0 ):
                result[idx] =  row + iter2[idx]
            else:
                result[idx] =  row - iter2[idx]
            result[self.pivot_row_idx, ] = new_row
        self.mat = result
        print("Iteration ", self.iteration , "\n---------------------\n", result)
    
    """ Get the pivot number , which is the cross section of the pivot row and the pivot column """
    def get_pivot_number(self):
        return(self.mat[ self.pivot_row_idx, self.pivot_column_idx])
  
    def isFinished(self):
        if(np.sum(self.mat[0, ] < 0) > 0):
            print('Not finished. Still zeros')
            return False
        else:
            print("Finished Optimization")
            return True
            
    
    def __check__(self):
        print("Pivot column is ", self.mat[:,self.pivot_column_idx])
        print("Pivot Row is  " , self.mat[self.pivot_row_idx, :])
        print("Pivot num is " , self.get_pivot_number())

        
    
if __name__ == '__main__':
    
    """
    Testing Function
    max Z = 3x + 5y
    st
    x = 4
    2y = 12
    3x + 2y = 18
    """
    
    np.set_printoptions(precision=2) # Clean precisions to make simplier
    #gnerate matrix (eventually to make simplier)
    mat = np.array([[1, -3, -5, 0, 0, 0, 0], [0, 1, 0, 1, 0 , 0, 4], [0, 0, 2, 0, 1 , 0, 12], [0, 3, 2, 0, 0, 1, 18]])
    print("Original Matrix to be Solved")
    print(mat)
    splx = SimplexMethod(mat)
    splx.solver()




Original Matrix to be Solved
[[ 1 -3 -5  0  0  0  0]
 [ 0  1  0  1  0  0  4]
 [ 0  0  2  0  1  0 12]
 [ 0  3  2  0  0  1 18]]
Not finished. Still zeros
----------------Print solving for  0
Iteration  1 
---------------------
 [[  1.   -3.    0.    0.    2.5   0.   30. ]
 [  0.    1.    0.    1.    0.    0.    4. ]
 [  0.    0.    1.    0.    0.5   0.    6. ]
 [  0.    3.    0.    0.   -1.    1.    6. ]]
Not finished. Still zeros
----------------Print solving for  1
Iteration  2 
---------------------
 [[  1.     0.     0.     0.     1.5    1.    36.  ]
 [  0.     0.     0.     1.     0.33  -0.33   2.  ]
 [  0.     0.     1.     0.     0.5    0.     6.  ]
 [  0.     1.     0.     0.    -0.33   0.33   2.  ]]
Finished Optimization
